IMPORTING REQUIRED LIBRARIES

In [0]:
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
__all__ = ['Inception3', 'inception_v3']

_InceptionOutputs = namedtuple('InceptionOutputs', ['logits', 'aux_logits'])

Mounted Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Change Path to Folder

In [0]:
import os
if not os.path.exists('/content/drive/My Drive/Inception_CIFAR10/'):
  os.makedirs('/content/drive/My Drive/Inception_CIFAR10/')
os.chdir('/content/drive/My Drive/Inception_CIFAR10/')

INCEPTION V3 FROM SCRATCH

In [0]:
def inception_v3(pretrained=False, **kwargs):
  
  if pretrained:
    if 'transform_input' not in kwargs:
      kwargs['transform_input'] = True
    if 'aux_logits' in kwargs:
      original_aux_logits = kwargs['aux_logits']
      kwargs['aux_logits'] = True
    else:
      original_aux_logits = True
    model = Inception3(**kwargs)
    if not original_aux_logits:
      model.aux_logits = False
    return model

  return Inception3(**kwargs)

In [0]:
class Inception3(nn.Module):

  def __init__(self, num_classes=10, aux_logits=True, transform_input=True):
    super(Inception3, self).__init__()
    self.aux_logits = aux_logits
    self.transform_input = transform_input
    self.Conv2d_4a_3x3 = BasicConv2d(3, 32, kernel_size=3,padding=1)
    self.Mixed_5b = InceptionA(32, pool_features=8)
    self.Mixed_5c = InceptionA(64, pool_features=72)
    self.Mixed_6a = InceptionB(128)
    self.Mixed_6b = InceptionC(256, channels_7x7=64)
    if aux_logits:
      self.AuxLogits = InceptionAux(512, num_classes)
    self.Mixed_7a = InceptionD(512)
    self.fc = nn.Linear(768, num_classes)

    for m in self.modules():
      if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        import scipy.stats as stats
        stddev = m.stddev if hasattr(m, 'stddev') else 0.1
        X = stats.truncnorm(-2, 2, scale=stddev)
        values = torch.as_tensor(X.rvs(m.weight.numel()), dtype=m.weight.dtype)
        values = values.view(m.weight.size())
        with torch.no_grad():
          m.weight.copy_(values)
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)

  def forward(self, x):
    global aux
    print(x.shape) 
    x = self.Conv2d_4a_3x3(x)
    x = self.Mixed_5b(x)
    x = self.Mixed_5c(x)
    x = self.Mixed_6a(x)
    x = self.Mixed_6b(x)
    if self.training and self.aux_logits:
      aux = self.AuxLogits(x)
    x = self.Mixed_7a(x)
    x = F.adaptive_avg_pool2d(x, (1, 1))
    x = F.dropout(x, training=self.training)
    x = torch.flatten(x, 1)
    x = self.fc(x)
    print(x.shape)
    if self.training and self.aux_logits:
      return _InceptionOutputs(x, aux)
    return x

In [0]:
class InceptionA(nn.Module):

  def __init__(self, in_channels, pool_features):
    super(InceptionA, self).__init__()
    self.branch1x1 = BasicConv2d(in_channels, 8, kernel_size=1)

    self.branch5x5_1 = BasicConv2d(in_channels, 8, kernel_size=1)
    self.branch5x5_2 = BasicConv2d(8, 16, kernel_size=5, padding=2)

    self.branch3x3dbl_1 = BasicConv2d(in_channels, 8, kernel_size=1)
    self.branch3x3dbl_2 = BasicConv2d(8, 16, kernel_size=3, padding=1)
    self.branch3x3dbl_3 = BasicConv2d(16, 32, kernel_size=3, padding=1)

    self.branch_pool = BasicConv2d(in_channels, pool_features, kernel_size=1)

  def forward(self, x):
    branch1x1 = self.branch1x1(x)

    branch5x5 = self.branch5x5_1(x)
    branch5x5 = self.branch5x5_2(branch5x5)

    branch3x3dbl = self.branch3x3dbl_1(x)
    branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
    branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

    branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
    branch_pool = self.branch_pool(branch_pool)

    outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
    return torch.cat(outputs, 1)

In [0]:
class InceptionB(nn.Module):

  def __init__(self, in_channels):
    super(InceptionB, self).__init__()
    self.branch3x3 = BasicConv2d(in_channels, 32, kernel_size=3, stride=2)

    self.branch3x3dbl_1 = BasicConv2d(in_channels, 32, kernel_size=1)
    self.branch3x3dbl_2 = BasicConv2d(32, 64, kernel_size=3, padding=1)
    self.branch3x3dbl_3 = BasicConv2d(64, 96, kernel_size=3, stride=2)

  def forward(self, x):
    branch3x3 = self.branch3x3(x)

    branch3x3dbl = self.branch3x3dbl_1(x)
    branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
    branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

    branch_pool = F.max_pool2d(x, kernel_size=3, stride=2)

    outputs = [branch3x3, branch3x3dbl, branch_pool]
    return torch.cat(outputs, 1)

In [0]:
class InceptionC(nn.Module):

  def __init__(self, in_channels, channels_7x7):
    super(InceptionC, self).__init__()
    self.branch1x1 = BasicConv2d(in_channels, 128, kernel_size=1)

    c7 = channels_7x7
    self.branch7x7_1 = BasicConv2d(in_channels, c7, kernel_size=1)
    self.branch7x7_2 = BasicConv2d(c7, c7, kernel_size=(1, 7), padding=(0, 3))
    self.branch7x7_3 = BasicConv2d(c7, 128, kernel_size=(7, 1), padding=(3, 0))

    self.branch7x7dbl_1 = BasicConv2d(in_channels, c7, kernel_size=1)
    self.branch7x7dbl_2 = BasicConv2d(c7, c7, kernel_size=(7, 1), padding=(3, 0))
    self.branch7x7dbl_3 = BasicConv2d(c7, c7, kernel_size=(1, 7), padding=(0, 3))
    self.branch7x7dbl_4 = BasicConv2d(c7, c7, kernel_size=(7, 1), padding=(3, 0))
    self.branch7x7dbl_5 = BasicConv2d(c7, 128, kernel_size=(1, 7), padding=(0, 3))

    self.branch_pool = BasicConv2d(in_channels, 128, kernel_size=1)

  def forward(self, x):
    branch1x1 = self.branch1x1(x)

    branch7x7 = self.branch7x7_1(x)
    branch7x7 = self.branch7x7_2(branch7x7)
    branch7x7 = self.branch7x7_3(branch7x7)

    branch7x7dbl = self.branch7x7dbl_1(x)
    branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
    branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
    branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
    branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)

    branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
    branch_pool = self.branch_pool(branch_pool)

    outputs = [branch1x1, branch7x7, branch7x7dbl, branch_pool]
    return torch.cat(outputs, 1)

In [0]:
class InceptionD(nn.Module):

  def __init__(self, in_channels):
    super(InceptionD, self).__init__()
    self.branch3x3_1 = BasicConv2d(in_channels, 32, kernel_size=1)
    self.branch3x3_2 = BasicConv2d(32,64, kernel_size=3, stride=2)

    self.branch7x7x3_1 = BasicConv2d(in_channels, 32, kernel_size=1)
    self.branch7x7x3_2 = BasicConv2d(32,64, kernel_size=(1, 7), padding=(0, 3))
    self.branch7x7x3_3 = BasicConv2d(64, 128, kernel_size=(7, 1), padding=(3, 0))
    self.branch7x7x3_4 = BasicConv2d(128,192, kernel_size=3, stride=2)

  def forward(self, x):
    branch3x3 = self.branch3x3_1(x)
    branch3x3 = self.branch3x3_2(branch3x3)

    branch7x7x3 = self.branch7x7x3_1(x)
    branch7x7x3 = self.branch7x7x3_2(branch7x7x3)
    branch7x7x3 = self.branch7x7x3_3(branch7x7x3)
    branch7x7x3 = self.branch7x7x3_4(branch7x7x3)

    branch_pool = F.max_pool2d(x, kernel_size=3, stride=2)
    outputs = [branch3x3, branch7x7x3, branch_pool]
    return torch.cat(outputs, 1)

In [0]:
class InceptionAux(nn.Module):

  def __init__(self, in_channels, num_classes):
    super(InceptionAux, self).__init__()
    self.conv0 = BasicConv2d(in_channels, 128, kernel_size=1)
    self.conv1 = BasicConv2d(128, 512, kernel_size=5)
    self.conv1.stddev = 0.01
    self.fc = nn.Linear(512, num_classes)
    self.fc.stddev = 0.001

  def forward(self, x):
    x = F.avg_pool2d(x, kernel_size=3, stride=3)
    x = self.conv0(x)
    x = self.conv1(x)
    x = F.adaptive_avg_pool2d(x, (1, 1))
    x = torch.flatten(x, 1)
    x = self.fc(x)
    return x

In [0]:
class BasicConv2d(nn.Module):

  def __init__(self, in_channels, out_channels, **kwargs):
    super(BasicConv2d, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
    self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

  def forward(self, x):
    x = self.conv(x)
    x = self.bn(x)
    return F.relu(x, inplace=True)

As suggested by you Sir, we removed some layers that were repeating in the architecture to reduce number of parameters to train

CODE FOR TRAINING

In [0]:
import os
import time

import torch
import torch.utils.data
import torchvision
from torchvision import transforms

# first train run this code
# import net

In [14]:
# incremental training comments out that line of code.

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

WORK_DIR = './data'
NUM_EPOCHS = 100
BATCH_SIZE = 32
#LEARNING_RATE = 0.01

MODEL_PATH = './model'
MODEL_NAME = 'Inception_v3.pth'

# Create model
if not os.path.exists(MODEL_PATH):
  os.makedirs(MODEL_PATH)

#AUGMENTATIONS
transform = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  #torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.4, saturation=0.5, hue=0.1),
  transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomVerticalFlip(),
  # torchvision.transforms.RandomAffine(degrees=0, translate=(0.2,0.2), scale=None,shear=50, resample=False, fillcolor=0),
  torchvision.transforms.RandomRotation((20), resample=False,expand=False, center=None),
  transforms.ToTensor(),
  transforms.Normalize([0.4913997551666284, 0.48215855929893703, 0.4465309133731618], [0.24703225141799082, 0.24348516474564, 0.26158783926049628])
])

# Load data
dataset = torchvision.datasets.CIFAR10(root=WORK_DIR,
                                        download=True,
                                        train=True,
                                        transform=transform)

dataset_loader = torch.utils.data.DataLoader(dataset=dataset,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True)


Files already downloaded and verified


In [15]:
# Total parameters
model = inception_v3().to(device)
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

2534260

FUNCTION TO START TRAINING

In [0]:
def main():
  print(f"Train numbers:{len(dataset)}")
  LEARNING_RATE = 0.001
  MOMENTUM=0.9
  # first train run this line
  #model = inception_v3().to(device)
  print(model)
  #model_save_name = 'Inception_v3e1.pth'
  model.load_state_dict(torch.load(MODEL_NAME))
  # Load model
  #if device == 'cuda':

    #model = torch.load(MODEL_PATH + MODEL_NAME).to(device)
  #else:
    #model = torch.load(MODEL_PATH + MODEL_NAME, map_location='cpu')
  # cast
  cast = torch.nn.CrossEntropyLoss().to(device)
  # Optimization
  optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LEARNING_RATE,
    momentum=MOMENTUM)
  step = 1
  loss_values=[]
  for epoch in range(1, NUM_EPOCHS + 1):
    print(loss_values)
    model.train()
    running_loss = 0.0
    
    # cal one epoch time
    start = time.time()
    correct = 0
    total = 0
    for images, labels in dataset_loader:
      images = images.to(device)
      print(images.shape)
      labels = labels.to(device)
      
      outputs, aux_outputs = model(images)
      loss1 = cast(outputs, labels)
      loss2 = cast(aux_outputs, labels)
      loss = loss1 + 0.4*loss2
      running_loss =+ loss.item() * images.size(0)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      print("epoch: ", epoch)
      print(f"Step [{step * BATCH_SIZE}/{NUM_EPOCHS * len(dataset)}], "
            f"Loss: {loss.item():.8f}.")
      print("Running Loss=",running_loss)
      step += 1
      # equal prediction and acc
      _, predicted = torch.max(outputs.data, 1)
      # val_loader total
      total += labels.size(0)
      # add correct
      correct += (predicted == labels).sum().item()

      print(f"Acc: {correct / total:.4f}.")
        # cal train one epoch time
    end = time.time()
    loss_values.append(running_loss / len(dataset_loader))
    
    print(f"Epoch [{epoch}/{NUM_EPOCHS}], "
          f"time: {end - start} sec!")

    # Save the model checkpoint
    if epoch%20==0:
    #   LEARNING_RATE=LEARNING_RATE/10
    #   torch.save(model, MODEL_PATH + '/' + MODEL_NAME)

      model_save_name = 'Inception_v3_CIFAR10_32BATCH_lr0.001_crop_bflip_rot'+str(epoch)+'.pth'   #WE keep changing this and saving states ,can be found in excel sheet attached
      torch.save(model.state_dict(), model_save_name)
    print("epoch completed and model copy completed")
    
  torch.save(model,MODEL_NAME)
  print(f"Model save to {MODEL_PATH + '/' + MODEL_NAME}.")

In [74]:
if __name__ == '__main__':
  main()

Streaming output truncated to the last 5000 lines.
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 10])
epoch:  96
Step [959296/1000000], Loss: 0.01983919.
Running Loss= 1.2697083950042725
Acc: 0.9987.
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 10])
epoch:  96
Step [959360/1000000], Loss: 0.01000556.
Running Loss= 0.6403558254241943
Acc: 0.9988.
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 10])
epoch:  96
Step [959424/1000000], Loss: 0.01426543.
Running Loss= 0.9129873514175415
Acc: 0.9988.
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 10])
epoch:  96
Step [959488/1000000], Loss: 0.01021658.
Running Loss= 0.6538611650466919
Acc: 0.9988.
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 10])
epoch:  96
Step [959552/1000000], Loss: 0.01176894.
Running Loss= 0.7532118558883667
Acc: 0.9988.
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 10

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Inception3. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type BasicConv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type InceptionA. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type InceptionB. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/

In [0]:
import os

import torch
import torch.utils.data
import torchvision
from torchvision import transforms


In [0]:
#!ls        #Saved model variations after every 20 epochs

CODE TO LOAD IN A MODEL CHECKPOINT FOR RETRAINING OR VALIDATION

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

WORK_DIR = './data'
BATCH_SIZE = 32

#MODEL_PATH = './model'
#MODEL_NAME = 'Inception_v3.pth'
#MODEL_NAME = "Inception_v3_CIFAR10_32SIZE_512BATCH_102_lr_low_para10.pth"
MODEL_NAME="Inception_v3_CIFAR10_32BATCH_lr0.001_crop_bflip_rot100.pth"


transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize([0.4913997551666284, 0.48215855929893703, 0.4465309133731618], [0.24703225141799082, 0.24348516474564, 0.26158783926049628])
])

In [20]:
# Load validation data
dataset = torchvision.datasets.CIFAR10(root=WORK_DIR,
                                        download=True,
                                        train=False,
                                        transform=transform)

dataset_loader = torch.utils.data.DataLoader(dataset=dataset,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True)

Files already downloaded and verified


CODE TO DO VALIDATION AND GET VALIDATION ACCURACY

In [0]:
def main():
  print(f"Val numbers:{len(dataset)}")
  #model = inception_v3().to(device)
  print(model)
  # Load model
  if device == 'cuda':
    #model = torch.load(MODEL_PATH+"/"+MODEL_NAME).to(device)
    model.load_state_dict(torch.load(MODEL_NAME))
  else:
    #model = torch.load(MODEL_PATH+"/"+MODEL_NAME, map_location='cpu')
    model.load_state_dict(torch.load(MODEL_NAME))
  model.eval()

  correct = 0.
  total = 0
  for images, labels in dataset_loader:
    # to GPU
    images = images.to(device)
    labels = labels.to(device)
    # print prediction
    outputs = model(images)
    # equal prediction and acc
    _, predicted = torch.max(outputs.data, 1)
    # val_loader total
    total += labels.size(0)
    # add correct
    correct += (predicted == labels).sum().item()

  print(f"Acc: {correct / total:.4f}.")


In [24]:
if __name__ == '__main__':
  
  main()

Val numbers:10000
Inception3(
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Mixed_5b): InceptionA(
    (branch1x1): BasicConv2d(
      (conv): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch5x5_1): BasicConv2d(
      (conv): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch5x5_2): BasicConv2d(
      (conv): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3dbl_1): BasicConv2d(
      (conv): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias

In [25]:
import numpy as np
total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)

print(f"Val numbers:{len(dataset)}")
#model = inception_v3().to(device)
print(model)
  # Load model
if device == 'cuda':
  #model = torch.load(MODEL_NAME).to(device)
  model.load_state_dict(torch.load(MODEL_NAME)).to(device)
else:
  #model = torch.load(MODEL_NAME, map_location='cpu')
  model.load_state_dict(torch.load(MODEL_NAME))
model.eval()
with torch.no_grad():
    for images,labels in dataset_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
print('Model accuracy on {0} test images: {1:.2f}%'.format(total_images, model_accuracy))

Val numbers:10000
Inception3(
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Mixed_5b): InceptionA(
    (branch1x1): BasicConv2d(
      (conv): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch5x5_1): BasicConv2d(
      (conv): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch5x5_2): BasicConv2d(
      (conv): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3dbl_1): BasicConv2d(
      (conv): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias

CATEGORY WISE VALIDATION ACCURACY

In [0]:
print('{0:10s} - {1}'.format('Category','Accuracy'))
for i, r in enumerate(confusion_matrix):
    print('{0:10s} - {1:.1f}'.format(classes[i], r[i]/np.sum(r)*100))

This helps us to analyse whether we need to go back and check the data for label mismatch

CONFUSION MATRIX

In [0]:
fig, ax = plt.subplots(1,1,figsize=(8,6))
ax.matshow(confusion_matrix, aspect='auto', vmin=0, vmax=1000, cmap=plt.get_cmap('Blues'))
plt.ylabel('Actual Category')
plt.yticks(range(10), classes)
plt.xlabel('Predicted Category')
plt.xticks(range(10), classes)
plt.show()

We ran a lot of different models ,tried dynamically changing learning rate after every 20 epochs,tried different combinations of augmentations. Decisions on how to conitnue further were made by looking at the loss.For example,if training loss looked unstable,we then reduced learning rate to prevent the model from bouncing around in parameter space. Similarly, by analysing loss, we took steps like adding regualrizers to prevent overfitting.
You can see all different variations in the excel sheet attached.